In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install dill
!pip install sentencepiece

In [4]:
import dill as pickle
import os
from tqdm import tqdm
import logging
import sentencepiece as spm

# Data preparation

In [ ]:
all_text = ""
for i in tqdm(os.listdir("/content/drive/MyDrive/GPT-2-Reproduction/Data/Preprocessed_texts")):
    with open(f"/content/drive/MyDrive/GPT-2-Reproduction/Data/Preprocessed_texts/{i}", "rb") as f:
        lst = pickle.load(f)
        all_text += " ".join(lst)

100%|██████████| 23/23 [01:38<00:00,  4.27s/it]


In [4]:
with open("/content/drive/MyDrive/GPT-2-Reproduction/Data/Full_Data.txt", 'r') as f:
    input = f.read()

In [5]:
with open(f"/content/drive/MyDrive/GPT-2-Reproduction/Data/Preprocessed_texts/sentences_0.pkl", "rb") as f:
    lst = pickle.load(f)

In [9]:
lst[1]

'სტატიის თანახმად, თვითაღიარებულ რესპუბლიკაში გამართულ არჩევნებში გაიმარჯვა იმ კანდიდატმა, რომლის საქმიანობასაც უკავშირდება რუსეთის შემადგენლობაში შესვლის მეტი იმედი.'

In [6]:
input[:10000]

'2017 წლის 19 აპრილი\nგზა აჩვენებს - ამ სათაურით გამოაქვეყნა კომერსანტმა სტატია, რომელშიც საუბარია როგორც ცხინვალში გამართული არჩევნებისა და რეფერენდუმის შესახებ, ასევე რუსეთის საგარეო საქმეთა მინისტრის ვიზიტზე სოხუმში. სტატიის თანახმად, თვითაღიარებულ რესპუბლიკაში გამართულ არჩევნებში გაიმარჯვა იმ კანდიდატმა, რომლის საქმიანობასაც უკავშირდება რუსეთის შემადგენლობაში შესვლის მეტი იმედი. რეალურად, არანაირი საარჩევნო პროგრამა, რომელშიც ამომრჩეველთა სოციალურ-ეკონომიკური მდგომარეობის გაუმჯობესების გეგმა იქნებოდა ჩამოყალიბებული, გამარჯვებულ კანდიდატს, ანატოლი ბიბილოვს, არ ჰქონია. მთელი იმედი დამყარებულია რუსეთთან მჭიდრო ინტეგრაციაზე, თუმცა, როგორც სტატიის ავტორი შენიშნავს, სამხრეთ ოსეთის მიღებას რუსეთის შემადგენლობაში მოჰყვება საერთაშორისო თანამეგობრობის ნეგატიური რეაქცია და ურთიერთობის სრული გაწყვეტა საქართველოსთან, მაშინ როცა ამ ურთიერთობაზე დამოკიდებულია რეგიონში რუსეთის ერთადერთი გეოპოლიტიკური მოკავშირის, სომხეთის, ბედი. რუსეთის სომხეთთან დამაკავშირებელი ერთადერთი სახმელეთო მარშრუტი სწორედ 

In [ ]:
with open("/content/drive/MyDrive/GPT-2-Reproduction/Data/Full_Data.txt", "w") as f:
    f.write(all_text)

# Train BPE Tokenizer Using Sentence Piece

In [5]:
options = dict(
  input="/content/drive/MyDrive/GPT-2-Reproduction/Data/Full_Data.txt",
  input_format="text",
  model_prefix="/content/drive/MyDrive/GPT-2-Reproduction/Models/bpe_tokenizer",
  model_type="bpe",
  vocab_size=32000,
  normalization_rule_name="identity",
  remove_extra_whitespaces=False,
  input_sentence_size=200000000,
  max_sentence_length=4192,
  seed_sentencepiece_size=1000000,
  shuffle_input_sentence=True,
  character_coverage=0.99995,
  byte_fallback=True,
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True,
  allow_whitespace_only_pieces=True,
  unk_id=0,
  bos_id=1,
  eos_id=2,
  pad_id=-1,
  num_threads=os.cpu_count(),
)
spm.SentencePieceTrainer.train(**options)

In [6]:
sp = spm.SentencePieceProcessor()
sp.load('/content/drive/MyDrive/GPT-2-Reproduction/Models/bpe_tokenizer.model')
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

In [12]:
encoded = sp.encode(lst[-1])

In [13]:
print([sp.id_to_piece(idx) for idx in encoded])

['▁როდესაც', '▁პატ', 'რის', '▁ევრა', '▁საბურთა', 'ოდ', '▁ტურ', 'ინში', '▁გადავიდა', ',', '▁მან', '▁ახალი', '▁თანაგუნ', 'დელები', '▁ოჯახში', '▁მიი', 'პატიჟ', 'ა', '▁და', '▁სანდრა', '▁ევ', 'რას', '▁ნახ', 'ელ', 'ავი', '▁კერძ', 'ებით', '▁თურმე', '▁ყველა', '▁კმაყოფილი', '▁და', '▁მადლი', 'ერი', '▁დარჩ', 'ენილა', '.']


In [14]:
print(lst[-1])
print(sp.decode(encoded))

როდესაც პატრის ევრა საბურთაოდ ტურინში გადავიდა, მან ახალი თანაგუნდელები ოჯახში მიიპატიჟა და სანდრა ევრას ნახელავი კერძებით თურმე ყველა კმაყოფილი და მადლიერი დარჩენილა.
როდესაც პატრის ევრა საბურთაოდ ტურინში გადავიდა, მან ახალი თანაგუნდელები ოჯახში მიიპატიჟა და სანდრა ევრას ნახელავი კერძებით თურმე ყველა კმაყოფილი და მადლიერი დარჩენილა.


In [15]:
for i in range(len(encoded)):
    print(f"{encoded[i]} --- {sp.decode(encoded[i])}")

1026 --- როდესაც
1247 --- პატ
444 --- რის
16653 --- ევრა
4613 --- საბურთა
376 --- ოდ
1724 --- ტურ
7983 --- ინში
8166 --- გადავიდა
31963 --- ,
702 --- მან
832 --- ახალი
17688 --- თანაგუნ
14852 --- დელები
5035 --- ოჯახში
2237 --- მიი
15041 --- პატიჟ
31944 --- ა
267 --- და
20098 --- სანდრა
720 --- ევ
2842 --- რას
802 --- ნახ
310 --- ელ
545 --- ავი
1770 --- კერძ
372 --- ებით
7256 --- თურმე
477 --- ყველა
11974 --- კმაყოფილი
267 --- და
8757 --- მადლი
356 --- ერი
1877 --- დარჩ
19992 --- ენილა
31968 --- .
